Student: Davide Sbetti - 14032

# Graphlab

First, let's import the graphlab library and pandas, used to export the final predictions

In [28]:
import graphlab as gl
import pandas as pd

We now read the movie dataset, using graphlab provided functions to avoid later conversions between different type of objects

In [29]:
movies = gl.SFrame.read_csv('data/train-PDA2019.csv')
print(movies.head())

Finished parsing file /home/davide/unibz/Master/Programming for Data Analytics/Jupyter/Project/data/train-PDA2019.csv

Parsing completed. Parsed 100 lines in 0.273906 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /home/davide/unibz/Master/Programming for Data Analytics/Jupyter/Project/data/train-PDA2019.csv

Parsing completed. Parsed 470711 lines in 0.325868 secs.

+--------+--------+--------+-----------+
| userID | itemID | rating | timeStamp |
+--------+--------+--------+-----------+
|   5    |  648   |   5    | 978297876 |
|   5    |  1394  |   5    | 978298237 |
|   5    |  3534  |   5    | 978297149 |
|   5    |  104   |   4    | 978298558 |
|   5    |  2735  |   5    | 978297919 |
|   5    |  3868  |   3    | 978298561 |
|   5    |  1079  |   5    | 978298384 |
|   5    |  2997  |   3    | 978298214 |
|   5    |  1615  |   5    | 978297755 |
|   5    |  1291  |   4    | 978297692 |
+--------+--------+--------+-----------+
[10 rows x 4 columns]



We now build the recommender system using the standard general function, which automatically identifies an appropriate model, based on the training data, and trains it

In [30]:
model = gl.recommender.create(movies, 'userID', 'itemID', 'rating', ranking = True)

Recsys training: model = ranking_factorization_recommender

Preparing data set.

Data has 470711 observations with 5690 users and 1824 items.

Data prepared in: 0.499028s

Training ranking_factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 1.42417

Final training RMSE: 0.884498

We now read the users test file, in order to get the list of users we are interested into and that we will use for the predictions

In [31]:
users_test = pd.read_csv("data/test-PDA2019.csv")
users_test.head()

,userID,recommended_itemIDs
0,1,
1,3,
2,11,
3,29,
4,31,


We now extract the column with the user IDs, in order to easily access them

In [32]:
users = users_test.loc[:,'userID']
users.head()

0     1
1     3
2    11
3    29
4    31
Name: userID, dtype: int64

We can now predict the rating for the movies that were not seen by the users. By default, Graphlab already returns the top 10 movies, ordered by their rating in decreasing order. We append the resulting formatted string to the pandas data frame, so that we can export the final predictions easily at the end

In [33]:
for j in range(0,len(users)):
    user = users[j]
    user_predictions = model.recommend([user])
    rec_string = " ".join(str(item) for item in user_predictions['itemID'])
    users_test.loc[j,'recommended_itemIDs'] = " " + rec_string

Having all predictions, we can now use pandas to export the complete data frame into a cvs file that can be submitted to the Kaggle platform for the evaluation

In [34]:
users_test.to_csv(path_or_buf = 'generated/graphlab_recommendations.csv', index = False, header = True, sep = ',')
print(users_test.head())

   userID                               recommended_itemIDs
0       1     2396 2762 318 1197 912 1270 260 1198 3147 593
1       3      1198 260 318 858 2396 2028 593 912 2762 1197
2      11      2762 260 593 318 2858 1196 3753 858 589 1270
3      29   593 1291 1270 2762 1198 590 1196 1197 2028 2396
4      31       2858 608 593 318 858 2028 2762 1617 223 589
